# Reinforced Anytime Bottom Up Rule Learning for Knowledge Graph Completion

## Abstract

Hầu hết các công việc hiện nay về việc hoàn thành đồ thị tri thức đều liên quan đến các phương pháp tiếp cận **sub-symbolic** (biểu tượng phụ) tập trung vào khái niệm nhúng một đồ thị đã cho trong một không gian vectơ có số chiều thấp. Ngược lại xu hướng này, chúng tôi đề xuất một cách tiếp cận được gọi là **AnyBURL** bắt nguồn từ ***symbolic space***(không gian tượng trưng). Thuật toán cốt lõi của nó dựa trên **sampling paths** các (lấy mẫu đường dẫn), được khái quát thành các **Horn rules**(quy tắc Horn). Các kết quả được công bố trước đây cho thấy chất lượng dự đoán của **AnyBURL** ngang bằng với tình trạng hiện tại của các phương pháp hiện đại *state of the art* với lợi ích bổ sung là đưa ra lời giải thích cho *predicted fact* (thực tế dự đoán). Trong bài báo này, chúng tôi quan tâm đến hai phần mở rộng của **AnyBURL**. Đầu tiên, chúng tôi thay đổi cách diễn giải các quy tắc của **AnyBURL** từ quy trình $\Theta$-subsumption thành $\Theta$-subsumption dưới *Object Identity*. Thứ hai, chúng tôi giới thiệu *reinforcement learning* (học tập củng cố) để hướng dẫn tốt hơn quá trình lấy mẫu. Chúng tôi phát hiện ra rằng học tăng cường giúp tìm ra các quy tắc có giá trị sớm hơn trong quá trình tìm kiếm. Chúng tôi đo lường tác động của cả hai phần mở rộng và so sánh cách tiếp cận kết quả với trạng thái hiện tại của phương pháp tiếp cận hiện đại. Kết quả của chúng tôi cho thấy **AnyBURL** vượt trội hơn hầu hết *sub-symbolic methods* (các phương pháp biểu tượng phụ).

## 1 Introduction

Đồ thị tri thức (KG) được sử dụng rộng rãi trong các lĩnh vực khác nhau. Ví dụ là FreeBase [2], DBPedia [1], YAGO [27], Google Sơ đồ tri thức và Microsoft Satori. Những KG khổng lồ này có thể chứa tới hàng triệu thực thể và hàng tỷ **fact** sự thật. Như đã chỉ ra trong [8], đồ thị tri thức thường không đầy đủ. Nhiệm vụ xây dựng bộ ba bị thiếu bằng cách sử dụng từ vựng đã được sử dụng trong đồ thị được gọi là hoàn thành đồ thị tri thức hoặc dự đoán liên kết. Nhiệm vụ này có thể được giải quyết với sự trợ giúp bổ sung của các tài nguyên bên ngoài (ví dụ: văn bản trong các trang web) hoặc bằng cách suy ra các bộ ba mới chỉ từ bộ ba trong một đồ thị kiến thức nhất định. Chúng tôi quan tâm đến vấn đề sau.

Một cách tiếp cận không sử dụng thông tin bên ngoài phải dựa vào số liệu thống kê, mẫu, phân phối hoặc bất kỳ loại tính đều đặn nào có thể tìm thấy trong đồ thị kiến thức đã cho. Một lựa chọn trực quan để giải quyết một nhiệm vụ như vậy là tìm hiểu và áp dụng một biểu diễn rõ ràng, mang tính biểu tượng của các mẫu này **symbolic representation**. Mặc dù có lịch sử lâu dài về các cách tiếp cận liên quan đến việc học *learning symbolic representations* các biểu tượng tượng trưng, chẳng hạn như lập trình logic quy nạp [19] và khai thác quy tắc quan hệ liên kết [6], ngày nay, nghiên cứu song song theo nhiều mô hình khác. Phần lớn các phương pháp được phát triển hiện nay học một biểu diễn phụ (biểu diễn ẫn), biểu tượng phụ của đồ thị tri thức đã cho. Lấy cảm hứng từ các mô hình ban đầu như RESCAL [20] và TransE [3], một số lượng lớn các mô hình mới đã được phát triển trong thập kỷ qua. Kết quả là, **symbolic approaches** (cách tiếp cận mang tính biểu tượng) được thể hiện trong nghiên cứu hoàn thành đồ thị tri thức.

### Cú pháp

***Atom*** *Kí hiệu mệnh đề (công thức nguyên tử): A, B, C,*

Các liên kết logic: $ \neg, \land, \lor, \to, \leftrightarrow$

Nếu $\alpha$ và $\beta$ là công thức thì các định nghĩa đệ qui đây cũng là công thức:
* Phủ định: $\neg \alpha$
* Nối liền (conjunction): $ \alpha \land \beta$
* Nối rời (disjunction): $ \alpha \lor \beta$
* Kéo theo (implication): $\alpha \to \beta$
* Tương đương (biconditional): $\alpha \leftrightarrow \beta$


### ví dụ
***Công thức:***
* $A$
* $\neg A$
* $\neg B \to C $
* $\neg A \land (\neg B \to C) \lor (\neg B \lor D)$
* $\neg \neg A $

***Không phải công thức:***
* $A \neg B$
* $A + B$

$KB$ ở dạng $CNF$ (Conjunctive Normal Form)

Môt literal hoặc là một công thức nguyên tử (postive literal) hoặc phủ định của công thức nguyên tử (negative literal)

Ví dụ: $A, \neg A, \dots$

Một clause (mệnh đề) là một literal hoặc **nối rờ**i của hai hoặc nhiều literal

Ví dụ: $A \lor \neg B \lor C$

Một công thức $\alpha$ được gọi là ở dạng $CNF$ nếu nó là **nối liền** của các clause
Ví dụ: $(A \lor B) \land (C \lor \neg D \lor E) \land (\neg E \lor F)$

[Horn rules](https://cs.nyu.edu/courses/spring03/G22.2560-001/horn.html)

### KB ở dạng Horn
Nếu các công thức trong KB được giới hạn ở dạng đặc biệt, một số luật suy diễn đúng có
thể trở thành đầy đủ

Chẳng hạn, dạng Horn (HNF – Horn normal form)

Hai luật suy diễn đúng và đầy đủ tương ứng với KB ở dạng HNF là:
* Hợp giải
* Tam đoạn luận (Modus ponens)

*Dạng Horn: một mệnh đề (clause) có nhiều nhất một positive literal*

Ví dụ: $ (A \lor \neg B) , ( \neg B \lor \neg C \lor D )$

***Lưu ý: Không phải mọi công thức trong logic mệnh đề đều có thể đưa về dạng Horn***

| Syntax               | Description |
|  :----               |    :----    |
| dạng                 | ¬B1 ∨ ¬B2 ∨. .∨ ¬Bk ∨ A hoặc B1 ∧ B2 ∧. .∧ Bk → A       |
| Facts                | Text        |
|Integrity Constraints |  222  |